See also: https://huggingface.co/docs/transformers/main/model_doc/t5

## Load Libraries

In [1]:
!pip install simplet5

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 51.5 MB/s eta 0:00:00
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6853 sha256=7ca576e7581b7093ea0bf705512962f58420c20b9e286b1cb4e64f376e89dcb2
  Stored in directory: /root/.cache/pip/wheels/b4/7d/af/743765400878438a7593f13f89fdf4004dcde0f2a8e6cb6684
Successfully built simplet5
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attemptin

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from simplet5 import SimpleT5


INFO:pytorch_lightning.utilities.seed:Global seed set to 42


## Load Data

In [3]:
def getSampleData():
  path = "https://raw.githubusercontent.com/Shivanandroy/T5-Finetuning-PyTorch/main/data/news_summary.csv"
  df = pd.read_csv(path)

  # simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
  df = df.rename(columns={"headlines":"target_text", "text":"source_text"})
  df = df[['source_text', 'target_text']]

  # T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
  df['source_text'] = "summarize: " + df['source_text']
  return df

In [4]:
df = getSampleData()


## Split Data

In [5]:
train_df, test_df = train_test_split(df, test_size=0.2)

## Create Model

In [6]:
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df,
            eval_df=test_df,
            source_max_token_len=128,
            target_max_token_len=50,
            batch_size=8, max_epochs=3, use_gpu=True)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


...

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [7]:
model.load_model("t5","outputs/epoch-2", use_gpu=True)

In [10]:
text_to_summarize="""summarize:
Lawyers for Alexey Navalny said Monday they have lost contact with the jailed Russian opposition leader, who was believed to be imprisoned in a penal colony about 150 miles east of Moscow, and his whereabouts are unknown.

Navalny was sentenced to 19 years in prison in August, after he was found guilty of creating an extremist community, financing extremist activities and numerous other crimes. He was already serving sentences of 11-and-a-half years in a maximum security facility on fraud and other charges he denies.

Supporters of Navalny claim his arrest and incarceration are a politically motivated attempt to stifle his criticism of Russian President Vladimir Putin.
"""
model.predict(text_to_summarize)

['Lawyers lose contact with jailed Russian opposition leader']

## Onyx

In [ ]:
#model.convert_and_load_onnx_model(model_dir="outputs/SimpleT5-epoch-2-train-loss-0.9526")
#model.onnx_predict(text_to_summarize)